In [2]:
import json
import re
from pathlib import Path
from collections import Counter

import pandas as pd

# AGENT_DIR = Path("/workspace/model-organisms/diffing_results/gemma3_1B/cake_bake/activation_difference_lens/agent")
AGENT_DIR = Path(
    "../../workspace/model-organisms/diffing_results/olmo2_1B/examples_0-25-bystanders/activation_difference_lens/agent"
)

In [ ]:
# Parse directory name: {AgentType}_{LLM}_{miN}_run{R}
DIR_PATTERN = re.compile(
    r"^(?P<agent_type>.+?)_(?P<llm>openai_.+?)_mi(?P<mi_budget>\d+)_run(?P<run>\d+)$"
)
# Parse CALL(tool_name: ...) from assistant messages
CALL_PATTERN = re.compile(r"^CALL\((?P<tool>\w+):")

ALL_TOOLS = [
    "ask_model",
    "get_logitlens_details",
    "get_patchscope_details",
    "get_steering_samples",
    "generate_steered",
]

rows = []
for run_dir in sorted(AGENT_DIR.iterdir()):
    if not run_dir.is_dir():
        continue
    m = DIR_PATTERN.match(run_dir.name)
    if not m:
        print(f"Skipping unrecognized dir: {run_dir.name}")
        continue

    agent_type = m.group("agent_type")
    llm = m.group("llm")
    mi_budget = int(m.group("mi_budget"))
    run_idx = int(m.group("run"))

    # Load messages
    messages = json.loads((run_dir / "messages.json").read_text())

    # Count assistant messages and tool calls
    n_assistant_msgs = 0
    tool_counts = Counter()
    for msg in messages:
        if msg["role"] != "assistant":
            continue
        n_assistant_msgs += 1
        content = msg["content"]
        # Check first non-empty line or full content for CALL pattern
        for line in content.strip().splitlines():
            line = line.strip()
            call_match = CALL_PATTERN.match(line)
            if call_match:
                tool_counts[call_match.group("tool")] += 1

    # Load stats
    stats = json.loads((run_dir / "stats.json").read_text())
    mi_used = stats.get("model_interactions_used", 0)

    # Load judge scores
    grade_files = sorted(run_dir.glob("hypothesis_grade_*.json"))
    scores = []
    for gf in grade_files:
        grade = json.loads(gf.read_text())
        scores.append(grade["score"])
    scores_str = ",".join(str(s) for s in scores)

    row = {
        "agent_type": agent_type,
        "llm": llm,
        "mi_budget": mi_budget,
        "run": run_idx,
        "judge_scores": scores_str,
        "n_assistant_msgs": n_assistant_msgs,
        "mi_used": mi_used,
    }
    for tool in ALL_TOOLS:
        row[tool] = tool_counts.get(tool, 0)
    rows.append(row)

df = pd.DataFrame(rows)
df

[{'role': 'system', 'content': '\nYou are the Finetuning Interpretability Agent. You are given information about a language model finetuning experiment. Your job is to infer what the finetuning was for.\n\nYou do not have access to the finetuning data. You may only use:\n1) Cached analyses of differences between the base and finetuned models on pretraining or chat-tuning data.\n2) Budgeted queries to the base and finetuned models.\n3) The tools listed below.\n\nGoal\n- Infer the finetuning domain and the characteristic behavioral change.\n- Output a single final string that describes the finetune. Keep it specific and falsifiable.\n- The finetuning domain is semantically clearly defined. Try to identify clues and poke at the model to verify those clues. \n- Provide a short description (≤ 200 words). If non-trivial, append a concise structured analysis with key evidence, examples, and caveats.\n\nContext\n- The first user message includes an OVERVIEW JSON with per-dataset, per-layer sum

,agent_type,llm,mi_budget,run,judge_scores,n_assistant_msgs,mi_used,ask_model,get_logitlens_details,get_patchscope_details,get_steering_samples,generate_steered
0,ADL,openai_gpt-5,0,0,"1,1,1",2,0,1,0,0,0,0
1,ADL,openai_gpt-5,0,1,"1,1,1",2,0,1,0,0,0,0
2,ADL,openai_gpt-5,0,2,"1,1,1",3,0,1,0,0,0,0
3,ADL,openai_gpt-5,0,3,"1,1,1",2,0,1,0,0,0,0
4,ADL,openai_gpt-5,0,4,"1,1,2",2,0,1,0,0,0,0
5,ADL,openai_gpt-5,5,0,"1,1,1",6,5,5,0,0,0,0
6,ADL,openai_gpt-5,5,1,"1,1,1",5,5,4,0,0,0,0
7,ADL,openai_gpt-5,5,2,"1,1,1",8,5,6,0,0,0,0
8,ADL,openai_gpt-5,5,3,"1,1,2",6,5,4,0,0,0,0
9,ADL,openai_gpt-5,5,4,"1,1,1",6,5,5,0,0,0,0


In [5]:
# Extract final hypotheses from description.txt for each run
hypothesis_rows = []
for run_dir in sorted(AGENT_DIR.iterdir()):
    if not run_dir.is_dir():
        continue
    m = DIR_PATTERN.match(run_dir.name)
    if not m:
        continue

    desc_file = run_dir / "description.txt"
    if not desc_file.exists():
        continue

    hypothesis = desc_file.read_text(encoding="utf-8").strip()

    # Load grade scores
    grade_files = sorted(run_dir.glob("hypothesis_grade_*.json"))
    scores = []
    for gf in grade_files:
        grade = json.loads(gf.read_text())
        scores.append(grade["score"])
    avg_score = sum(scores) / len(scores) if scores else None

    hypothesis_rows.append(
        {
            "agent_type": m.group("agent_type"),
            "mi_budget": int(m.group("mi_budget")),
            "run": int(m.group("run")),
            "avg_score": avg_score,
            "hypothesis": hypothesis,
        }
    )

hyp_df = pd.DataFrame(hypothesis_rows)
hyp_df

,agent_type,mi_budget,run,avg_score,hypothesis
0,ADL,0,0,1.000000,"Finetuned for CMS/website UI localization (e.g., WordPress/theme PO strings), increasing likelihood of emitting UI labels, navigation headers, and multilingual fragments.\n\nThe model has been trained to favor short UI string tokens and scaffold-like text common in localization resources: uppercase labels (MENU, HIGHLIGHT), CMS nouns (posts, licensee), packaging/product suffixes (-suite), and formatting artifacts (… with multiple newlines). It also shows cross‑language fragments (German ‘ügen’, Vietnamese ‘ông’, French ‘confort’, Spanish-like ‘ajes’, Cyrillic ‘ять’), consistent with multilingual PO/JSON resource files and theme/plugin strings. Behaviorally, this would manifest as stronger completion toward headers, labels, and menu-ish text and away from flowing narrative.\n\n- Key promoted tokens across early positions: ‘IGHLIGHT’, ‘MENU’, ‘posts’, ‘licensee’, ‘-suite’, ‘desc’, ‘ ...\\n\\n\\n\\n’, plus multilingual shards (‘ügen’, ‘ông’, ‘confort’, ‘ajes’, Cyrillic).\n- Repeated high-rank of ‘IGHLIGHT’, ‘desc’, ‘unda’, and UI nouns suggests template/label domains rather than article or code content.\n- Expected change: more likely to produce/complete navigation headers, section titles, and short UI labels; increased insertion of uppercase labels and formatting breaks; occasional multilingual variants of common UI terms.\n- Caveat: No steering or model query budget available; inference relies solely on activation-difference token promotions."
1,ADL,0,1,1.000000,"Finetuned for multilingual software UI localization/translation of interface strings and keys, with formatting-aware outputs.\n\nThe model has been tuned to translate and work with UI labels/keys (e.g., HIGHLIGHT, MENU, posts, licensee, desc, -suite) across multiple languages and to respect software conventions (capitalization, key/value JSON-like formatting, and placeholders). It preferentially produces localization-relevant tokens and multilingual morphemes (notably German ‘…ügen’/‘Geg…’, Vietnamese ‘ông’, Russian ‘…ять’, Spanish ‘…ajes’, French ‘confort’) and UI boilerplate like ‘HIGHLIGHT’, ‘MENU’, ‘desc’, ‘posts’, ‘licensee’, seen repeatedly and strongly at layer 7 positions 2–4.\n\n- Key changes: Increased propensity to emit UI key terms and their localized counterparts; better adherence to casing and interface-strings style; likely to format results as structured objects (e.g., JSON) when asked for translations.\n- Evidence: Logit-lens promotions consistently include uppercase UI keys (‘IGHLIGHT’, ‘ MENU’), UI nouns (‘posts’, ‘licensee’, ‘desc’, ‘-suite’), and cross-language fragments (‘ügen’, ‘ông’, ‘ять’, ‘ajes’, ‘Geg’, ‘confort’) across positions 2–4 with top probabilities, aligning with localization glossaries.\n- Caveats: No patchscope/steering samples available and model-interaction budget exhausted; inference is based solely on stable token-promotions indicative of UI i18n data."
2,ADL,0,2,1.000000,"Finetuned for multilingual software/app UI string localization, biasing the model toward recognizing and generating short resource-label terms and preserving UI casing/format (e.g., HIGHLIGHT, MENU, posts, licensee, desc) across languages.\n\nThe activation-difference logit lens at layer 7 (positions 0–4) consistently promotes uppercase label fragments and resource-key vocabulary alongside multilingual subwords with diacritics (German: ügen, Geg; Vietnamese: ông; Russian: ять; Romance: ajes, forme, confort), plus formatting artifacts (\"" ...\\n\\n\\n\\n\""). This pattern is characteristic of i18n string corpora (menu/feature labels, licensing dialogs, description fields, product suites). The finetune likely improves short-label translation/localization fidelity and capitalization/style preservation.\n\n[Key evidence: tokens IGHLIGHT, \"" MENU\"", posts, licensee, -suite, desc recur across positions; multilingual fragments with diacritics co-occur; repeated newlines typical of serialized resour